## Python Code 1: Extract Strings from EXE

In this step I wrote a Python script to **scan the EXE file**.  
The goal was to **find hidden text** inside the binary file.


1. Open the file `Test.exe` in **binary mode**.  
2. Extract **ASCII strings** and **UTF-16 strings**.  
3. Filter results with **keywords** like "SystemCode", "Register", "License".  
4. Also search for **key formats** (like XXXX-XXXX-XXXX).  
5. Save all results to:
   - `all_strings.txt` → all strings  
   - `hits.txt` → only the suspicious ones  


- It helps me see what kind of text the program keeps inside.  
- Some of these strings can show **how registration works**.  


In [ ]:
import re

file_path = "Test.exe"

def extract_ascii_strings(data, min_len=4):
    result, buf, pos, start = [], [], [], None
    for i, b in enumerate(data):
        if 32 <= b <= 126:
            if start is None:
                start = i
            buf.append(chr(b))
        else:
            if len(buf) >= min_len:
                result.append((start, "".join(buf)))
            buf, start = [], None
    if len(buf) >= min_len:
        result.append((start, "".join(buf)))
    return result

def extract_utf16le_strings(data, min_len=4):
    result, buf, start = [], [], None
    for i in range(0, len(data)-1, 2):
        lo, hi = data[i], data[i+1]
        if hi == 0 and 32 <= lo <= 126:
            if start is None:
                start = i
            buf.append(chr(lo))
        else:
            if len(buf) >= min_len:
                result.append((start, "".join(buf)))
            buf, start = [], None
    if len(buf) >= min_len:
        result.append((start, "".join(buf)))
    return result

# 載入檔案
with open(file_path, "rb") as f:
    data = f.read()

ascii_strs = extract_ascii_strings(data)
utf16_strs = extract_utf16le_strings(data)

all_strs = ascii_strs + utf16_strs

# 關鍵字過濾 
keywords = [
    "SystemCode","SysCode","SystemID","MachineID","HWID",
    "License","Licence","LicenseKey","ProductKey","Serial","Activation",
    "Register","Key","RegCode","AuthCode","AppKey","Token","Code",
    "系統碼","序號","註冊","激活","授權","系統代碼","序列號"
]

hits = []
for offset, s in all_strs:
    for k in keywords:
        if k.lower() in s.lower():
            hits.append((offset, s))
            break

# 疑似金鑰格式
pattern = re.compile(r"[A-Z0-9]{4,6}(?:-[A-Z0-9]{4,6}){2,6}")
for offset, s in all_strs:
    for m in pattern.findall(s):
        hits.append((offset, m))

# 去重
hits = sorted(set(hits), key=lambda x: x[0])

# 輸出 
with open("all_strings.txt", "w", encoding="utf-8") as f:
    for offset, s in all_strs:
        f.write(f"[0x{offset:08X}] {s}\n")

with open("hits.txt", "w", encoding="utf-8") as f:
    for offset, s in hits:
        f.write(f"[0x{offset:08X}] {s}\n")

print(f"完成掃描！共找到 {len(all_strs)} 筆字串")
print(f"全部字串輸出到 all_strings.txt")
print(f"可疑字串輸出到 hits.txt")


完成掃描！共找到 2045 筆字串
全部字串輸出到 all_strings.txt
可疑字串輸出到 hits.txt


## Python Code 2: Get CPU Information

In this step I used Python with the **wmi** library.  
The goal was to read the **CPU system code** from Windows.

### What the code does
1. Connect to Windows WMI.  
2. Get information from `Win32_Processor`.  
3. Print:
   - `DeviceID`
   - `ProcessorId`
   - `Name`

### Why this is useful
- The program uses the **ProcessorId** to build the **system code**.  
- With this script I can check what value my computer gives.  
- It also helps in a simple **security risk check**,  
  because if the code only depends on CPU ID, it may be weak.  


In [3]:
import wmi

c = wmi.WMI()
for cpu in c.Win32_Processor():
    print("DeviceID:", cpu.DeviceID)
    print("ProcessorId:", cpu.ProcessorId)
    print("Name:", cpu.Name)


DeviceID: CPU0
ProcessorId: BFEBFBFF000806D1
Name: 11th Gen Intel(R) Core(TM) i7-11800H @ 2.30GHz


## Python Code 3: Write Registry Key

In this step I used Python with the **winreg** library.  
The goal was to test how the program saves the **registration status**.

### What the code does
1. Open (or create) the key:  
   `HKEY_CURRENT_USER\Software\BES_Report`  
2. Set the value `RegStatus` to `"1"`.  
3. Close the key and print a message.

### Why this is useful
- The program checks this key to see if it is registered.  
- By setting it to `"1"`, I can simulate a **registered state**.  
- This is part of the test to learn how the software works.  


In [4]:
import winreg

key = winreg.CreateKey(winreg.HKEY_CURRENT_USER, r"Software\BES_Report")
winreg.SetValueEx(key, "RegStatus", 0, winreg.REG_SZ, "1")
winreg.CloseKey(key)
print("RegStatus 已設為 1")


RegStatus 已設為 1
